In [34]:

import pandas as pd
import requests

In [35]:

# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text

In [39]:

def fetch_player_stats_single_year(year: int):
    df = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'))[0]
    df_adv = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'))[0]

    df = df.drop(df[df['Player'] == 'Player'].index)

    df_adv = df_adv.drop(df_adv[df_adv['Player'] == 'Player'].index)

    new = pd.merge(df, df_adv, on=['Player', 'Age', 'Tm', 'Pos'])

    player_stats_list = ['Player', 'Pos', 'Age', 'Tm', 'G_x', 'GS', 'MP_x', 'FG', 'FGA',
                         'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
                         'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                         'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
                         'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
                         'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

    player_stats_list_correct = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
                                 '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
                                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER',
                                 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
                                 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
                                 'VORP']

    # cuts down the columns to what is listed in player_stats_list_correct
    new = new[player_stats_list]
    new.columns = player_stats_list_correct

    # This gets rid of traded players entirely
    # new = new.drop_duplicates(subset='Player', keep=False).reset_index(drop=True)
    new.drop_duplicates(subset=['Player'], keep='first', inplace=True)
    # Fills nan's with 0
    new.fillna(value=0, inplace=True)

    # Adds a year column
    new['Year'] = year

    return new

    # if not os.path.isdir(f'Data/Test Sets/{datetime.now().strftime("%Y")}'):
    #     os.mkdir(f'Data/Test Sets/{datetime.now().strftime("%Y")}')
    #
    # new.to_csv(f'Data/Test Sets/{datetime.now().strftime("%Y")}/{datetime.now().strftime("%Y%m%d")}_player_stats.csv')
    # print(datetime.now().strftime("%m/%d/%Y"), 'Complete')




In [37]:
player_list = []
for i in range(1980, 2023):
    print(f'Done - {i}')
    player_list.append(fetch_player_stats_single_year(i))


df = pd.concat(player_list)

Done - 1980
Done - 1981
Done - 1982
Done - 1983
Done - 1984
Done - 1985
Done - 1986
Done - 1987
Done - 1988
Done - 1989
Done - 1990
Done - 1991
Done - 1992
Done - 1993
Done - 1994
Done - 1995
Done - 1996
Done - 1997
Done - 1998
Done - 1999
Done - 2000
Done - 2001
Done - 2002
Done - 2003
Done - 2004
Done - 2005
Done - 2006
Done - 2007
Done - 2008
Done - 2009
Done - 2010
Done - 2011
Done - 2012
Done - 2013
Done - 2014
Done - 2015
Done - 2016
Done - 2017
Done - 2018
Done - 2019
Done - 2020
Done - 2021
Done - 2022


In [38]:
df.shape

(22467, 50)

In [12]:


# df['All_Star'] = df.Player.str.contains('\*')
#
df.Player = df.Player.str.strip('\*')
# df['All_Star']


0       True
1      False
2      False
3       True
4      False
       ...  
805    False
808    False
809    False
810    False
811    False
Name: All_Star, Length: 18264, dtype: bool

In [32]:
import numpy as np

# df.All_Star = np.where(df.All_Star == True,False,True)


df[df.Player == 'Stephen Curry']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,All_Star
120,Stephen Curry,PG,21,GSW,80,77,36.2,6.6,14.3,0.462,...,3.0,1.6,4.7,0.077,1.1,-0.5,0.7,2.0,2010,True
153,Stephen Curry,PG,22,GSW,74,74,33.6,6.8,14.2,0.480,...,5.4,1.3,6.6,0.128,3.0,-0.7,2.3,2.7,2011,True
109,Stephen Curry,PG,23,GSW,26,23,28.2,5.6,11.4,0.490,...,1.8,0.4,2.2,0.144,4.1,0.3,4.3,1.2,2012,True
119,Stephen Curry,PG,24,GSW,78,78,38.2,8.0,17.8,0.451,...,8.4,2.8,11.2,0.180,5.3,0.1,5.4,5.6,2013,True
149,Stephen Curry,PG,25,GSW,78,78,36.5,8.4,17.7,0.471,...,9.3,4.0,13.4,0.225,6.3,1.1,7.4,6.7,2014,True
135,Stephen Curry,PG,26,GSW,80,80,32.7,8.2,16.8,0.487,...,11.5,4.1,15.7,0.288,8.2,1.7,9.9,7.9,2015,True
116,Stephen Curry,PG,27,GSW,79,79,34.2,10.2,20.2,0.504,...,13.8,4.1,17.9,0.318,10.3,1.6,11.9,9.5,2016,True
120,Stephen Curry,PG,28,GSW,79,79,33.4,8.5,18.3,0.468,...,8.7,3.9,12.6,0.229,6.7,0.3,6.9,5.9,2017,True
146,Stephen Curry,PG,29,GSW,51,51,32.0,8.4,16.9,0.495,...,7.2,1.9,9.1,0.267,7.8,0.0,7.7,4.0,2018,True
159,Stephen Curry,PG,30,GSW,69,69,33.8,9.2,19.4,0.472,...,7.2,2.5,9.7,0.199,7.1,-0.5,6.6,5.1,2019,True


In [15]:
df2 = df.copy()

In [16]:
df2[(df2.Year == 1987) & (df2.Player == 'Michael Jordan')]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,All_Star
167,Michael Jordan,SG,23,CHI,82,82,40.0,13.4,27.8,.482,...,11.9,5.0,16.9,.247,8.4,2.4,10.8,10.6,1987,1


In [17]:

# For some reason the 1997 MVP file doesn't contain Karl (winner) or Michael (2nd) by default. They were added to the file manually.

awards_df = list()
for year in range(1980, 2023):
    awards_df.append(
        pd.read_csv(f'/Users/chaseallbright/Desktop/content/Cleaned Awards/MVP/{year}-Most Valuable Player.csv',
                    index_col=0))

awards_df = pd.concat(awards_df)

In [18]:
awards_df[(awards_df.MVP_Player == 'Karl Malone') & (awards_df.Year == 1997)]

,MVP_Rank,MVP_Player,MVP_Age,MVP_First,MVP_Pts Won,MVP_Pts Max,MVP_Share,Year
0,1,Karl Malone,33,1.0,986.0,1150,0.857,1997


In [19]:
df = df2.copy()

df.loc[:, ['Player', 'Pos', 'Tm']] = df[['Player', 'Pos', 'Tm']].astype('string')
df.loc[:, ['Age', 'G', 'GS', 'Year']] = df[['Age', 'G', 'GS', 'Year']].astype('int')
df.loc[:, df.drop(columns=(['Age', 'G', 'GS', 'Year', 'Player', 'Pos', 'Tm'])).columns] = df.loc[:, df.drop(
    columns=(['Age', 'G', 'GS', 'Year', 'Player', 'Pos', 'Tm'])).columns].astype('float64')

In [20]:
full = pd.merge(df, awards_df, right_on=['MVP_Player', 'MVP_Age'], left_on=['Player', 'Age'], how='left')
full = full.drop(columns=['Year_y', 'MVP_Player', 'MVP_Age'], )

In [21]:
full = full.fillna(0)

In [22]:
full.isna().sum()

Player         0
Pos            0
Age            0
Tm             0
G              0
GS             0
MP             0
FG             0
FGA            0
FG%            0
3P             0
3PA            0
3P%            0
2P             0
2PA            0
2P%            0
eFG%           0
FT             0
FTA            0
FT%            0
ORB            0
DRB            0
TRB            0
AST            0
STL            0
BLK            0
TOV            0
PF             0
PTS            0
PER            0
TS%            0
3PAr           0
FTr            0
ORB%           0
DRB%           0
TRB%           0
AST%           0
STL%           0
BLK%           0
TOV%           0
USG%           0
OWS            0
DWS            0
WS             0
WS/48          0
OBPM           0
DBPM           0
BPM            0
VORP           0
Year_x         0
All_Star       0
MVP_Rank       0
MVP_First      0
MVP_Pts Won    0
MVP_Pts Max    0
MVP_Share      0
dtype: int64

In [23]:
full.Pos.unique()

<StringArray>
[       'C',       'PF',       'PG',       'SG',       'SF',    'SG-PG',
    'SF-SG',    'SG-SF',     'C-PF',     'PF-C',    'SF-PF',    'PG-SG',
    'PF-SF',    'PG-SF',    'SG-PF',     'SF-C', 'SG-PG-SF']
Length: 17, dtype: string

In [24]:
pos_map = {'PG': 1,
           'SG': 2,
           'SF': 3,
           'PF': 4,
           'C': 5,
           'SG-SF': 2,
           'SG-PG': 2,
           'PG-SG': 1,
           'SF-PF': 3,
           'PG-SF': 1,
           'SG-PF': 2,
           'SF-C': 3,
           'SF-SG': 3,
           'PF-SF': 4,
           'C-PF': 5,
           'PF-C': 4,
           'SG-PG-SF': 2,
           }

full.Pos = full.Pos.map(pos_map).astype('int')

In [25]:
full.Pos.unique()

array([5, 4, 1, 2, 3])

In [26]:
full['Year_x'].unique()


array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [33]:
pd.read_csv('Full Dataset.csv',index_col=0).drop(columns='All_Star',inplace=True)

full.to_csv('Full Dataset.csv')

In [22]:
full[(full.Year_x == 1997) & (full.MVP_Share != 0)]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,OBPM,DBPM,BPM,VORP,Year_x,MVP_Rank,MVP_First,MVP_Pts Won,MVP_Pts Max,MVP_Share
5982,Charles Barkley,4,33,HOU,53,53,37.9,6.3,13.1,0.484,...,4.5,1.2,5.7,3.9,1997,16.0,0.0,2.0,1150.0,0.002
6080,Patrick Ewing,5,34,NYK,78,78,37.0,8.4,17.2,0.488,...,0.9,1.9,2.8,3.5,1997,8.0,0.0,57.0,1150.0,0.050
6109,Tom Gugliotta,4,27,MIN,81,81,38.7,7.3,16.5,0.442,...,1.2,0.5,1.7,2.9,1997,17.0,0.0,1.0,1150.0,0.001
6116,Tim Hardaway,1,30,MIA,81,81,38.7,7.1,17.1,0.415,...,5.2,1.0,6.2,6.5,1997,4.0,0.0,238.0,1150.0,0.207
6128,Grant Hill,3,24,DET,80,80,39.3,7.8,15.7,0.496,...,5.4,2.7,8.2,8.0,1997,3.0,0.0,376.0,1150.0,0.327
6138,Allen Iverson,1,21,PHI,76,74,40.1,8.2,19.8,0.416,...,2.5,-1.6,0.9,2.2,1997,17.0,0.0,1.0,1150.0,0.001
6147,Kevin Johnson,1,30,PHO,70,70,38.0,6.3,12.7,0.496,...,5.3,-0.3,5.0,4.7,1997,17.0,0.0,1.0,1150.0,0.001
6152,Michael Jordan,2,33,CHI,82,82,37.9,11.2,23.1,0.486,...,7.8,1.1,8.9,8.6,1997,2.0,0.0,957.0,1150.0,0.832
6188,Karl Malone,4,33,UTA,82,82,36.6,10.5,19.2,0.550,...,7.1,1.2,8.3,7.8,1997,1.0,1.0,986.0,1150.0,0.857
6199,Anthony Mason,4,30,CHH,73,73,43.1,5.9,11.3,0.525,...,2.6,0.8,3.5,4.3,1997,9.0,0.0,7.0,1150.0,0.006
